In [1]:
import sys
sys.path.append('/mnt/c/Users/Olive/python_projects/FinNLP/FinNLP')  # https://github.com/AI4Finance-Foundation/FinNLP

In [2]:
# only for WSL
import os 
os.environ["PATH"] = f"{os.environ['PATH']}:/usr/local/cuda-12.0/bin"
# os.environ['LD_LIBRARY_PATH'] = "/usr/lib/wsl/lib:/usr/local/cuda/lib64"
os.environ['LD_LIBRARY_PATH'] = "/usr/local/cuda-12.0/lib64"
os.environ["BNB_CUDA_VERSION"]="120"

In [3]:
from transformers import AutoModel, AutoTokenizer, AutoModelForCausalLM, LlamaForCausalLM, LlamaTokenizerFast   # 4.30.2
from peft import PeftModel  # 0.4.0
import torch

from finnlp.benchmarks.fpb import test_fpb
from finnlp.benchmarks.fiqa import test_fiqa , add_instructions
from finnlp.benchmarks.tfns import test_tfns
from finnlp.benchmarks.nwgi import test_nwgi

/home/oliverwang15/miniconda3/lib/python3.10/site-packages/bitsandbytes-0.41.1-py3.10.egg/bitsandbytes/cuda_setup/main.py:106: UserWarning: 

BNB_CUDA_VERSION=XXX can be used to load a bitsandbytes version that is different from the PyTorch CUDA version.
If this was unintended set the BNB_CUDA_VERSION variable to an empty string: export BNB_CUDA_VERSION=
If you use the manual override make sure the right libcudart.so is in your LD_LIBRARY_PATH
For example by adding the following to your .bashrc: export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:<path_to_cuda_dir/lib64
Loading CUDA version: BNB_CUDA_VERSION=120


  warn((f'\n\n{"="*80}\n'


## Load Model (Pick one according the model from the following blocks)

#### 1. **FInGPT v3.1** based on ChatGLM2, runable on 1 * RTX 3090

In [4]:
# # v3.1
# base_model = "THUDM/chatglm2-6b"
# peft_model = "oliverwang15/FinGPT_v31_ChatGLM2_Sentiment_Instruction_LoRA_FT"
# tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
# model = AutoModel.from_pretrained(base_model, trust_remote_code=True, load_in_8bit = True, device_map = "auto")
# model = PeftModel.from_pretrained(model, peft_model)
# model = model.eval()

#### 2. **FinGPT v3.2** based on Llama2, runable on 1 * A 100 and also runable on 1 * RTX 3090 as long as `load_in_8bit = True` is set in Line 8, but the speed is slower

In [5]:
# v3.2
# base_model = "meta-llama/Llama-2-7b-chat-hf"  # Access needed
base_model = "NousResearch/Llama-2-13b-hf" 
peft_model = "oliverwang15/FinGPT_v33_Llama2_13B_Sentiment_Instruction_LoRA_FT_8bit"
tokenizer = LlamaTokenizerFast.from_pretrained(base_model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
model = LlamaForCausalLM.from_pretrained(base_model, trust_remote_code=True, device_map = "cuda:0", load_in_8bit = True,)
model = PeftModel.from_pretrained(model, peft_model)
model = torch.compile(model)  # Please comment this line if your platform does not support torch.compile
model = model.eval()

The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

## Test

In [ ]:
batch_size = 16

In [ ]:
# FPB
res = test_fpb(model, tokenizer, batch_size = batch_size)



Prompt example:
Instruction: What is the sentiment of this news? Please choose an answer from {negative/neutral/positive}.
Input: L&T has also made a commitment to redeem the remaining shares by the end of 2011 .
Answer: 


Total len: 1212. Batchsize: 16. Total steps: 76


100%|██████████| 76/76 [01:10<00:00,  1.08it/s]

Acc: 0.8820132013201321. F1 macro: 0.8774649459784639. F1 micro: 0.8820132013201321. F1 weighted (BloombergGPT): 0.8817589259286243. 


In [ ]:
def pf(x):
    return  "What is the sentiment of this news? Please choose an answer from {strong negative/moderately negative/mildly negative/neutral/mildly positive/moderately positive/strong positive}, then provide some short reasons."

In [ ]:
# FiQA
res = test_fiqa(model, tokenizer, prompt_fun = add_instructions, batch_size = batch_size)
# res = test_fiqa(model, tokenizer, batch_size = batch_size, prompt_fun=pf)



Prompt example:
Instruction: What is the sentiment of this tweet? Please choose an answer from {negative/neutral/positive}.
Input: This $BBBY stock options trade would have more than doubled your money https://t.co/Oa0loiRIJL via @TheStreet
Answer: 


Total len: 275. Batchsize: 16. Total steps: 18


100%|██████████| 18/18 [00:14<00:00,  1.25it/s]

Acc: 0.8581818181818182. F1 macro: 0.7693085447271324. F1 micro: 0.858181818181818. F1 weighted (BloombergGPT): 0.8744065786191484. 


In [ ]:
# TFNS
res = test_tfns(model, tokenizer, batch_size = batch_size)



Prompt example:
Instruction: What is the sentiment of this tweet? Please choose an answer from {negative/neutral/positive}.
Input: $ALLY - Ally Financial pulls outlook https://t.co/G9Zdi1boy5
Answer: 


Total len: 2388. Batchsize: 16. Total steps: 150


100%|██████████| 150/150 [01:56<00:00,  1.29it/s]

Acc: 0.9032663316582915. F1 macro: 0.8787781808799305. F1 micro: 0.9032663316582915. F1 weighted (BloombergGPT): 0.9028238287011249. 


In [ ]:
# NWGI
res = test_nwgi(model, tokenizer, batch_size = batch_size)



Prompt example:
Instruction: What is the sentiment of this news? Please choose an answer from {negative/neutral/positive}.
Input: In the latest trading session, Adobe Systems (ADBE) closed at $535.98, marking a +0.31% move from the previous day.
Answer: 


Total len: 4047. Batchsize: 16. Total steps: 253


100%|██████████| 253/253 [04:38<00:00,  1.10s/it]

Acc: 0.6429453916481345. F1 macro: 0.6546976499843747. F1 micro: 0.6429453916481345. F1 weighted (BloombergGPT): 0.6428106146054466. 
